In [18]:
import collections
SentimentDocument = collections.namedtuple('SentimentDocument', 'words tags split sentiment')

In [19]:
import logging
logging.basicConfig(format='%(asctime)s | %(levelname)s | - %(message)s', level=logging.INFO)

In [20]:
# import io
# import re
# import tarfile
# import os.path

# import smart_open
# import gensim.utils

# def download_dataset(url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'):
#     fname = url.split('/')[-1]

#     if os.path.isfile(fname):
#        return fname

#     # Download the file to local storage first.
#     with smart_open.open(url, "rb", ignore_ext=True) as fin:
#         with smart_open.open(fname, 'wb', ignore_ext=True) as fout:
#             while True:
#                 buf = fin.read(io.DEFAULT_BUFFER_SIZE)
#                 if not buf:
#                     break
#                 fout.write(buf)

#     return fname

# def create_sentiment_document(name, text, index):
#     _, split, sentiment_str, _ = name.split('/')
#     sentiment = {'pos': 1.0, 'neg': 0.0, 'unsup': None}[sentiment_str]

#     if sentiment is None:
#         split = 'extra'

#     tokens = gensim.utils.to_unicode(text).split()
#     return SentimentDocument(tokens, [index], split, sentiment)

# def extract_documents(file_path):

#     index = 0

#     with tarfile.open(file_path, mode='r:gz') as tar:
#         for member in tar.getmembers():
#             if re.match(r'aclImdb/(train|test)/(pos|neg|unsup)/\d+_\d+.txt$', member.name):
#                 member_bytes = tar.extractfile(member).read()
#                 member_text = member_bytes.decode('utf-8', errors='replace')
#                 assert member_text.count('\n') == 0
#                 yield create_sentiment_document(member.name, member_text, index)
#                 index += 1

In [21]:
# # fname = download_dataset()
# # alldocs = list(extract_documents(filename))
# tar_data_path = "./data/aclImdb_v1.tar.gz"
# alldocs = list(extract_documents(tar_data_path))

In [22]:
# import torch
# data_path = "./data/imdb.data"
# torch.save(alldocs, data_path)

In [23]:
import torch
data_path = "./data/imdb.data"
imdb_docs = torch.load(data_path)

In [24]:
train_docs = [doc for doc in imdb_docs if doc.split == 'train']
test_docs = [doc for doc in imdb_docs if doc.split == 'test']
print(f'{len(imdb_docs)} docs: {len(train_docs)} train-sentiment, {len(test_docs)} test-sentiment')

100000 docs: 25000 train-sentiment, 25000 test-sentiment


In [25]:
import numpy as np
import statsmodels.api as sm
from random import sample

def logistic_predictor_from_data(train_targets, train_regressors):
    """Fit a statsmodel logistic predictor on supplied data"""
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets = [doc.sentiment for doc in train_set]
    train_regressors = [test_model.dv[doc.tags[0]] for doc in train_set]
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_regressors = [test_model.dv[doc.tags[0]] for doc in test_set]
    test_regressors = sm.add_constant(test_regressors)

    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_set])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

In [26]:
from gensim.models.callbacks import CallbackAny2Vec

class EpochEval(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
        print(f"Epoch {self.epoch} - Error Rate: {err_rate} | Model: {model}")
        self.epoch+=1

In [27]:
import multiprocessing

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

given_kwargs = dict(vector_size=200, window=5, min_count=5)

common_kwargs = dict(workers=9,
                     epochs=20)

hs_kwargs = dict(hs=1, negative=0)
ns_kwargs = dict(hs=0, negative=5)
dm_kwargs = dict(dm=1) #, alpha=0.5, min_alpha=0.0002)
dbow_kwargs = dict(dm=0) #, alpha=0.25, min_alpha=0.0001)

models = [
    Doc2Vec(**hs_kwargs, **dm_kwargs, **given_kwargs, **common_kwargs),
    Doc2Vec(**hs_kwargs, **dbow_kwargs, **given_kwargs, **common_kwargs),
    Doc2Vec(**ns_kwargs, **dm_kwargs, **given_kwargs, **common_kwargs),
    Doc2Vec(**ns_kwargs, **dbow_kwargs, **given_kwargs, **common_kwargs),
]

2024-05-08 13:25:11,524 | INFO | - Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>', 'datetime': '2024-05-08T13:25:11.524881', 'gensim': '4.3.2', 'python': '3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) \n[GCC 12.3.0]', 'platform': 'Linux-4.15.0-213-generic-x86_64-with-glibc2.27', 'event': 'created'}
2024-05-08 13:25:11,527 | INFO | - Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow,d200,hs,mc5,s0.001,t9>', 'datetime': '2024-05-08T13:25:11.527129', 'gensim': '4.3.2', 'python': '3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) \n[GCC 12.3.0]', 'platform': 'Linux-4.15.0-213-generic-x86_64-with-glibc2.27', 'event': 'created'}
2024-05-08 13:25:11,528 | INFO | - Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d200,n5,w5,mc5,s0.001,t9>', 'datetime': '2024-05-08T13:25:11.528663', 'gensim': '4.3.2', 'python': '3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) \n[GCC 12.3.0]', 'platform': 'Linux-4.15.0-213-generic-x86

In [28]:
from concurrent.futures import ProcessPoolExecutor

def build_vocal_task(model, docs):
    model.build_vocab(docs)
    return model

with ProcessPoolExecutor() as executor:
    futures = []
    for model in models:
        future = executor.submit(build_vocal_task, model, imdb_docs)
        futures.append(future)
    models = [future.result() for future in futures]

2024-05-08 13:25:37,835 | INFO | - collecting all words and their counts
2024-05-08 13:25:37,840 | INFO | - PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2024-05-08 13:25:38,521 | INFO | - PROGRESS: at example #10000, processed 2292381 words (3375741 words/s), 150816 word types, 0 tags
2024-05-08 13:25:39,040 | INFO | - PROGRESS: at example #20000, processed 4573645 words (4404808 words/s), 238497 word types, 0 tags
2024-05-08 13:25:39,640 | INFO | - PROGRESS: at example #30000, processed 6865575 words (3832953 words/s), 312348 word types, 0 tags
2024-05-08 13:25:40,260 | INFO | - PROGRESS: at example #40000, processed 9190019 words (3763556 words/s), 377231 word types, 0 tags
2024-05-08 13:25:40,856 | INFO | - PROGRESS: at example #50000, processed 11557847 words (3983235 words/s), 438729 word types, 0 tags
2024-05-08 13:25:41,437 | INFO | - PROGRESS: at example #60000, processed 13899883 words (4048555 words/s), 493913 word types, 0 tags
2024-05-08 13:2

In [29]:
from random import shuffle
shuffled_imdbdocs = imdb_docs[:]
shuffle(shuffled_imdbdocs)
import time

def train_task(model):
    start_time = time.time()
    print(f"Training {model}\n")
    model.train(shuffled_imdbdocs,
                total_examples=len(shuffled_imdbdocs),
                epochs=model.epochs,
                callbacks=[EpochEval()])
    print(f"Training {model} took {time.time() - start_time} seconds")
    model_name = f"{model}"
    model_name = model_name.replace("/", "_")
    torch.save(model, f"./data/{model_name}.model")
    return model

with ProcessPoolExecutor() as executor:
    futures = []
    for model in models:
        future = executor.submit(train_task, model)
        futures.append(future)
    models = [future.result() for future in futures]

Training Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>



2024-05-08 13:26:39,352 | INFO | - Doc2Vec lifecycle event {'msg': 'training model with 9 workers on 120726 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 negative=0 window=5 shrink_windows=True', 'datetime': '2024-05-08T13:26:39.352469', 'gensim': '4.3.2', 'python': '3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) \n[GCC 12.3.0]', 'platform': 'Linux-4.15.0-213-generic-x86_64-with-glibc2.27', 'event': 'train'}
2024-05-08 13:26:40,429 | INFO | - EPOCH 0 - PROGRESS: at 1.71% examples, 285283 words/s, in_qsize 17, out_qsize 0
2024-05-08 13:26:41,430 | INFO | - EPOCH 0 - PROGRESS: at 3.49% examples, 301730 words/s, in_qsize 17, out_qsize 0


Training Doc2Vec<dbow,d200,hs,mc5,s0.001,t9>



2024-05-08 13:26:41,836 | INFO | - Doc2Vec lifecycle event {'msg': 'training model with 9 workers on 120726 vocabulary and 200 features, using sg=1 hs=1 sample=0.001 negative=0 window=5 shrink_windows=True', 'datetime': '2024-05-08T13:26:41.835841', 'gensim': '4.3.2', 'python': '3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) \n[GCC 12.3.0]', 'platform': 'Linux-4.15.0-213-generic-x86_64-with-glibc2.27', 'event': 'train'}


Training Doc2Vec<dm/m,d200,n5,w5,mc5,s0.001,t9>



...

Epoch 18 - Error Rate: 0.19272 | Model: Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>


2024-05-08 13:47:13,672 | INFO | - EPOCH 19 - PROGRESS: at 1.58% examples, 256959 words/s, in_qsize 17, out_qsize 0
2024-05-08 13:47:14,675 | INFO | - EPOCH 19 - PROGRESS: at 3.45% examples, 293995 words/s, in_qsize 18, out_qsize 0
2024-05-08 13:47:15,706 | INFO | - EPOCH 19 - PROGRESS: at 5.31% examples, 304335 words/s, in_qsize 17, out_qsize 0
2024-05-08 13:47:16,735 | INFO | - EPOCH 19 - PROGRESS: at 7.19% examples, 309282 words/s, in_qsize 18, out_qsize 0
2024-05-08 13:47:17,764 | INFO | - EPOCH 19 - PROGRESS: at 8.96% examples, 309654 words/s, in_qsize 17, out_qsize 0
2024-05-08 13:47:18,764 | INFO | - EPOCH 19 - PROGRESS: at 10.82% examples, 312631 words/s, in_qsize 17, out_qsize 0
2024-05-08 13:47:19,771 | INFO | - EPOCH 19 - PROGRESS: at 12.74% examples, 315417 words/s, in_qsize 17, out_qsize 0
2024-05-08 13:47:20,781 | INFO | - EPOCH 19 - PROGRESS: at 14.50% examples, 313717 words/s, in_qsize 18, out_qsize 0
2024-05-08 13:47:21,786 | INFO | - EPOCH 19 - PROGRESS: at 16.34% exa

Epoch 19 - Error Rate: 0.19272 | Model: Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>


2024-05-08 13:48:14,660 | INFO | - Doc2Vec lifecycle event {'msg': 'training on 465590580 raw words (353030093 effective words) took 1295.3s, 272546 effective words/s', 'datetime': '2024-05-08T13:48:14.660282', 'gensim': '4.3.2', 'python': '3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) \n[GCC 12.3.0]', 'platform': 'Linux-4.15.0-213-generic-x86_64-with-glibc2.27', 'event': 'train'}


Training Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9> took 1295.3169384002686 seconds


In [30]:
from collections import defaultdict
error_rates = defaultdict(lambda: 1.0)  # To selectively print only best errors achieved

In [31]:
for model in models:
    print(f"Evaluating {model}\n")
    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
    error_rates[str(model)] = err_rate
    print(f"{err_rate} {model}\n")

Evaluating Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>

0.19272 Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>

Evaluating Doc2Vec<dbow,d200,hs,mc5,s0.001,t9>

0.12592 Doc2Vec<dbow,d200,hs,mc5,s0.001,t9>

Evaluating Doc2Vec<dm/m,d200,n5,w5,mc5,s0.001,t9>

0.15848 Doc2Vec<dm/m,d200,n5,w5,mc5,s0.001,t9>

Evaluating Doc2Vec<dbow,d200,n5,mc5,s0.001,t9>

0.10392 Doc2Vec<dbow,d200,n5,mc5,s0.001,t9>



In [32]:
print("Err_rate Model")
for rate, name in sorted((rate, name) for name, rate in error_rates.items()):
    print(f"{rate} {name}")

Err_rate Model
0.10392 Doc2Vec<dbow,d200,n5,mc5,s0.001,t9>
0.12592 Doc2Vec<dbow,d200,hs,mc5,s0.001,t9>
0.15848 Doc2Vec<dm/m,d200,n5,w5,mc5,s0.001,t9>
0.19272 Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>


In [33]:
print("Accuracy Model")
for rate, name in sorted((rate, name) for name, rate in error_rates.items()):
    print(f"{(1-rate)*100:.2f} {name}")

Accuracy Model
89.61 Doc2Vec<dbow,d200,n5,mc5,s0.001,t9>
87.41 Doc2Vec<dbow,d200,hs,mc5,s0.001,t9>
84.15 Doc2Vec<dm/m,d200,n5,w5,mc5,s0.001,t9>
80.73 Doc2Vec<dm/m,d200,hs,w5,mc5,s0.001,t9>
